## Vanessa Williams
## Week 9

In [ ]:
!pip install scikit-learn spacy sklearn-crfsuite
!python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 24.2 -> 24.3
[notice] To update, run: pip install --upgrade pip
zsh:1: command not found: python


In [ ]:
import pandas as pd

# Specify the path to the file; adjust the path if needed
file_path = "/Users/vanessawilliams/Desktop/Vanessa_Williams/ner.csv"

# Load the data
data = pd.read_csv(file_path)

# Display the first few rows of the data
data.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [ ]:
import ast

# Convert 'POS' and 'Tag' columns from strings to lists
data['POS'] = data['POS'].apply(ast.literal_eval)
data['Tag'] = data['Tag'].apply(ast.literal_eval)

# Verify conversion by displaying sample data
print(data[['Sentence', 'POS', 'Tag']].head())

                                            Sentence   
0  Thousands of demonstrators have marched throug...  \
1  Families of soldiers killed in the conflict jo...   
2  They marched from the Houses of Parliament to ...   
3  Police put the number of marchers at 10,000 wh...   
4  The protest comes on the eve of the annual con...   

                                                 POS   
0  [NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...  \
1  [NNS, IN, NNS, VBN, IN, DT, NN, VBD, DT, NNS, ...   
2  [PRP, VBD, IN, DT, NNS, IN, NN, TO, DT, NN, IN...   
3  [NNS, VBD, DT, NN, IN, NNS, IN, CD, IN, NNS, V...   
4  [DT, NN, VBZ, IN, DT, NN, IN, DT, JJ, NN, IN, ...   

                                                 Tag  
0  [O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...  
1  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...  
2  [O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo...  
3      [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]  
4  [O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,..

#### Preparing data for CRF training

In [ ]:
sentences = []
for _, row in data.iterrows():
    sentence = []
    words = row['Sentence'].split()
    pos_tags = row['POS']
    ner_tags = row['Tag']

    for word, pos, ner in zip(words, pos_tags, ner_tags):
        sentence.append((word, pos, ner))
    sentences.append(sentence)

# Display a sample sentence to verify
print(sentences[0])

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'postag': postag,
    }

    # Add surrounding words as context
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True  # Beginning of sentence

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True  # End of sentence

    return features


In [ ]:
# Prepare features and labels for CRF model
X = [sentence2features(s) for s in sentences]
y = [sentence2labels(s) for s in sentences]

In [ ]:
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report

# Initialize and train the CRF model
crf = CRF(algorithm='lbfgs', c1=0.01, c2=0.01, max_iterations=200)

crf.fit(X, y)

CRF(algorithm='lbfgs', c1=0.01, c2=0.01, max_iterations=200)

In [ ]:
test_sentence = "Fourteen days ago, Emperor Palpatine left San Diego, CA for Tatooine to follow Luke Skywalker."
test_tokens = test_sentence.split()

# Preprocess and extract features for the test sentence
test_features = [word2features([(token, '') for token in test_tokens], i) for i in range(len(test_tokens))]
# Predict tags using the trained model
crf_predictions = crf.predict([test_features])[0]

# Display results
print("CRF Model Prediction:")
for token, tag in zip(test_tokens, crf_predictions):
    print(f"{token}: {tag}")

CRF Model Prediction:
Fourteen: O
days: O
ago,: B-art
Emperor: I-art
Palpatine: I-art
left: I-art
San: I-art
Diego,: I-art
CA: I-art
for: I-art
Tatooine: I-art
to: I-art
follow: I-art
Luke: I-art
Skywalker.: I-art


#### Test with spaCy

In [ ]:
import spacy

# Load spaCy's pre-trained NER model
nlp = spacy.load("en_core_web_sm")

# Input sentence
sentence = "Fourteen days ago, Emperor Palpatine left San Diego, CA for Tatooine to follow Luke Skywalker."

# Run sentence through spaCy NER
doc = nlp(sentence)

# Display spaCy's NER output
for ent in doc.ents:
    print(ent.text, ent.label_)

Fourteen days ago DATE
Palpatine PERSON
San Diego GPE
CA WORK_OF_ART
Tatooine PERSON
Luke Skywalker PERSON


### Comparison of Custom CRF NER Model and spaCy NER Model

**1. Custom CRF NER Model Output:**
- **Output:**  
- Fourteen: O
- days: O
- ago,: B-art
- Emperor: I-art
- Palpatine: I-art
- left: I-art
- San: I-art
- Diego,: I-art
- CA: I-art
- for: I-art
- Tatooine: I-art
- to: I-art
- follow: I-art
- Luke: I-art
- Skywalker.: I-art

- **Observations:**  
The CRF model tagged most words as `I-art` (artistic work), which doesn’t accurately capture the intended entities. This suggests the CRF model may have overfit on certain tags, leading to generalized labeling as `art`.

**2. spaCy NER Model Output:**
- **Output:**
- Fourteen days ago - DATE
- Palpatine - PERSON
- San Diego - GPE (Geopolitical Entity)
- CA - WORK_OF_ART
- Tatooine - PERSON
- Luke Skywalker - PERSON

- **Observations:**  
spaCy accurately labeled most entities:
- Recognized "Fourteen days ago" as a `DATE`.
- Correctly identified "Palpatine" and "Luke Skywalker" as `PERSON`.
- Tagged "San Diego" as `GPE`, which is appropriate.
- Misinterpreted "CA" as `WORK_OF_ART`, likely due to context ambiguity.

**3. Comparison Summary:**
 - **Accuracy**: spaCy’s model is more accurate for this sentence, correctly identifying `DATE`, `PERSON`, and `GPE`.
 - **Model Training**: The custom CRF model may need additional training data or parameter tuning to improve its classification, as it’s consistently tagging entities as `art`.
 - **spaCy Strengths**: spaCy benefits from extensive pre-training, which helps it recognize common entities even in fictional contexts (like "Luke Skywalker").
 - **CRF Limitations**: The CRF model requires well-labeled, domain-specific training data to improve its accuracy.

This comparison demonstrates that spaCy’s pre-trained NER is better suited for general-purpose NER tasks, while custom NER models require careful tuning and training to achieve similar accuracy.